In [1]:
import numpy as np
# we want to simulate walking around the cliff in miniature world with both sarsa and q-learning

In [2]:
# Algorithm parameters: step size ↵ 2 (0, 1], small " > 0
# Initialize Q(s, a), for all s 2 S+, a 2 A(s), arbitrarily except that Q(terminal , ·) = 0
# Loop for each episode:
    # Initialize S
    # Loop for each step of episode:
        # Choose A from S using policy derived from Q (e.g., "-greedy)
        # Take action A, observe R, S0
        # Q(S,A) Q(S,A) + ↵
        # ⇥
        # R + # maxa Q(S', a) − Q(S,A)
        # S⇤S'
        # 
    # until S is terminal

In [3]:
# actions

def epsilon_greedy_action(state: int, q_table: np.array, epsilon: float) -> int:
    """
    Select action based on the ε-greedy policy
    Random action with prob. ε, greedy action with prob. 1-ε
    """

    # Random uniform sample from [0,1]
    sample = np.random.random()

    # Set to 'explore' if sample <= ε
    explore = True if sample <= epsilon else False

    if explore:  # Explore
        # Select random action
        action = np.random.choice(4)
    else:  # Exploit:
        # Select action with largest Q-value
        action = np.argmax(q_table[:, state])

    return action


def move_agent(agent_pos: tuple, action: int) -> tuple:
    """
    Move agent to new position based on current position and action
    """
    # Retrieve agent position
    (pos_x, pos_y) = agent_pos

    if action == 0:  # Up
        pos_x = pos_x - 1 if pos_x > 0 else pos_x
    elif action == 1:  # Down
        pos_x = pos_x + 1 if pos_x < 3 else pos_x
    elif action == 2:  # Left
        pos_y = pos_y - 1 if pos_y > 0 else pos_y
    elif action == 3:  # Right
        pos_y = pos_y + 1 if pos_y < 11 else pos_y
    else:  # Infeasible move
        raise Exception("Infeasible move")

    agent_pos = (pos_x, pos_y)

    return agent_pos

def get_max_qvalue(state: int, q_table: np.array) -> float:
    """Retrieve best Q-value for state from table"""
    maximum_state_value = np.amax(q_table[:, state])
    return maximum_state_value


def get_reward(state: int, cliff_pos: np.array, goal_pos: int) -> int:
    """
    Compute reward for given state
    """

    # Reward of -1 for each move (including terminating)
    reward = -0.1

    # Reward of +100 for reaching goal
    if state == goal_pos:
        reward = 100

    # Reward of -100 for falling down cliff
    if state in cliff_pos:
        reward = -100

    return reward

def compute_cum_rewards(gamma: float, t: int, rewards: np.array) -> float:
    """Cumulative reward function"""
    cum_reward = 0
   # cum_reward = rewards[-1] #TEST!!!
    for tau in range(t, len(rewards)):
        cum_reward += gamma ** (tau - t) * rewards[tau]
    return cum_reward

In [4]:
# q table updates

def init_q_table(x_dim: int = 12, y_dim: int = 4) -> np.array:
    """
    Initialize Q-table to store values state-action pairs
    Set Q(s, a) = 0, for all s ∈ S, a ∈ A(s)
    """
    # Initialize Q-table (4 actions per state) with zeros
    q_table = np.zeros((4, x_dim * y_dim))

    return q_table


def update_q_table(
    q_table: np.array,
    state: int,
    action: int,
    reward: int,
    next_state_value: float,
    gamma: float,
    alpha: float,
) -> np.array:
    """
    Update Q-table based on observed rewards and next state value
    For SARSA (on-policy):
    Q(S, A) <- Q(S, A) + [α * (r + (γ * Q(S', A'))) -  Q(S, A)]
    For Q-learning (off-policy):
    Q(S, A) <- Q(S, A) + [α * (r + (γ * max(Q(S', A*)))) -  Q(S, A)
    """
    # Compute new q-value
    new_q_value = q_table[action, state] + alpha * (
        reward + (gamma * next_state_value) - q_table[action, state]
    )

    # Replace old Q-value
    q_table[action, state] = new_q_value

    return q_table

In [5]:
# Environment

""" Tile layout (36=start, 47=goal, 37-46=cliff)
0	1	2	3	4	5	6	7	8	9	10	11
12	13	14	15	16	17	18	19	20	21	22	23
24	25	26	27	28	29	30	31	32	33	34	35
36	37	38	39	40	41	42	43	44	45	46	47
"""

def check_game_over(episode:int,
    state: int, cliff_pos: np.array, goal_pos: int, number_of_steps: int
) -> bool:
    """
    Function returns reward in the given state
    """
    # Game over when reached goal, fell down cliff, or exceeded 1000 steps
    game_over = (
        True
        if (state == goal_pos   or   state in cliff_pos or number_of_steps == 99)
        else False
    )
    if state == goal_pos and number_of_steps > 1:
        print("===== Goal reached (episode", episode ,") =====")

    return game_over


def init_env() -> (tuple, np.array, np.array, int, bool):
    """Initialize environment and agent position"""
    agent_pos = (3, 0)  # Left-bottom corner (start)
    env = np.zeros((4, 12), dtype=int)
    env = mark_path(agent_pos, env)
    cliff_states = np.arange(37, 47)  # States for cliff tiles
    goal_state = 47  # State for right-bottom corner (destination)
    game_over = False

    return agent_pos, env, cliff_states, goal_state, game_over


def mark_path(agent: tuple, env: np.array) -> np.array:
    """
    Store path taken by agent
    Only needed for visualization
    """
    (posY, posX) = agent
    env[posY][posX] += 1

    return env


def env_to_text(env: np.array) -> str:
    """
    Convert environment to text format
    Needed for visualization in console
    """
    env = np.where(env >= 1, 1, env)

    env = np.array2string(env, precision=0, separator=" ", suppress_small=False)
    env = env.replace("[[", " |")
    env = env.replace("]]", "|")
    env = env.replace("[", "|")
    env = env.replace("]", "|")
    env = env.replace("1", "x")
    env = env.replace("0", " ")

    return env


def encode_vector(index: int, dim: int) -> list:
    """Encode vector as one-hot vector"""
    vector_encoded = np.zeros((1, dim))
    vector_encoded[0, index] = 1

    return vector_encoded



def get_state(agent_pos: tuple) -> int:
    """
    Obtain state corresponding to agent position
    """
    x_dim = 12
    (pos_x, pos_y) = agent_pos
    state = x_dim * pos_x + pos_y

    return state


def get_position(state: int) -> tuple:
    """
    Obtain agent position corresponding to state
    """
    x_dim = 12

    pos_x = int(np.floor(state / x_dim))
    pos_y = state % x_dim

    agent_pos = (pos_x, pos_y)

    return agent_pos

In [7]:
def qlearning(sim_input, sim_output) -> (np.array, list):
    """
    Q-learning algorithm
    """
    num_episodes = sim_input.num_episodes
    gamma = sim_input.gamma
    alpha = sim_input.alpha
    epsilon = sim_input.epsilon

    q_table = init_q_table()
    steps_cache = np.zeros(num_episodes)
    rewards_cache = np.zeros(num_episodes)

    # Iterate over episodes
    for episode in range(num_episodes):

        # Set to target policy at final episode
        if episode == len(range(num_episodes)) - 1:
            epsilon = 0

        # Initialize environment and agent position
        agent_pos, env, cliff_pos, goal_pos, game_over = init_env()
        num_steps = 0

        while not game_over:
            # Get state corresponding to agent position
            state = get_state(agent_pos)

            # Select action using ε-greedy policy
            action = epsilon_greedy_action(state, q_table, epsilon)

            # Move agent to next position
            agent_pos = move_agent(agent_pos, action)

            # Mark visited path
            env = mark_path(agent_pos, env)

            # Determine next state
            next_state = get_state(agent_pos)

            # Compute and store reward
            reward = get_reward(next_state, cliff_pos, goal_pos)
            rewards_cache[episode] += reward

            # Check whether game is over
            game_over = check_game_over(episode, next_state, cliff_pos, goal_pos, num_steps)

            # Determine maximum Q-value next state (off-policy)
            max_qvalue_next_state = get_max_qvalue(next_state, q_table)

            # Update Q-table
            q_table = update_q_table(
                q_table, state, action, reward, max_qvalue_next_state, gamma, alpha
            )

            num_steps += 1

        steps_cache[episode] = num_steps

    sim_output.step_cache.append(steps_cache)
    sim_output.reward_cache.append(rewards_cache)

    sim_output.env_cache.append(env)
    sim_output.name_cache.append("Q-learning")

    return q_table, sim_output

In [8]:
"""Learn cliff walking policies with SARSA and Q-learning"""
# Set input parameters
class sim_init:
    def __init__(self, num_episodes, gamma, alpha, epsilon):
        self.num_episodes = num_episodes  # Number of training episodes
        self.gamma = gamma  # Discount rate γ 0.9
        self.alpha = alpha  # Learning rate α 0.001
        self.epsilon = epsilon  # Exploration rate ε

    def __str__(self):
        return "# episodes: " + str(self.num_episodes) + "gamma: " + str(self.gamma) \
               + "alpha: " + str(self.alpha) + "epsilon: " + str(self.epsilon)

run_algorithms = {
       "Q-Learning",
#       "SARSA",
#   "Monte Carlo",
#      "Discrete policy gradient",
#      "Deep policy gradient",
#      "Deep Q-Learning",
    }
class sim_output:
        def __init__(self, rewards_cache, step_cache, env_cache, name_cache):
            self.reward_cache = rewards_cache  # list of rewards
            self.step_cache = step_cache  # list of steps
            self.env_cache = env_cache  # list of final paths
            self.name_cache = name_cache  # list of algorithm names

sim_output = sim_output(
    rewards_cache=[], step_cache=[], env_cache=[], name_cache=[]
)
    
if "Q-Learning" in run_algorithms:
    sim_input = sim_init(num_episodes=10000, gamma=1.0, alpha=0.01, epsilon=0.05)
    (qtable_qlearning, sim_output) = qlearning(sim_input, sim_output)

===== Goal reached (episode 25 ) =====
===== Goal reached (episode 30 ) =====
===== Goal reached (episode 37 ) =====
===== Goal reached (episode 44 ) =====
===== Goal reached (episode 46 ) =====
===== Goal reached (episode 56 ) =====
===== Goal reached (episode 57 ) =====
===== Goal reached (episode 60 ) =====
===== Goal reached (episode 64 ) =====
===== Goal reached (episode 68 ) =====
===== Goal reached (episode 70 ) =====
===== Goal reached (episode 74 ) =====
===== Goal reached (episode 75 ) =====
===== Goal reached (episode 78 ) =====
===== Goal reached (episode 79 ) =====
===== Goal reached (episode 80 ) =====
===== Goal reached (episode 83 ) =====
===== Goal reached (episode 85 ) =====
===== Goal reached (episode 86 ) =====
===== Goal reached (episode 90 ) =====
===== Goal reached (episode 92 ) =====
===== Goal reached (episode 93 ) =====
===== Goal reached (episode 95 ) =====
===== Goal reached (episode 100 ) =====
===== Goal reached (episode 101 ) =====
===== Goal reached (epi

===== Goal reached (episode 650 ) =====
===== Goal reached (episode 651 ) =====
===== Goal reached (episode 652 ) =====
===== Goal reached (episode 653 ) =====
===== Goal reached (episode 654 ) =====
===== Goal reached (episode 655 ) =====
===== Goal reached (episode 656 ) =====
===== Goal reached (episode 657 ) =====
===== Goal reached (episode 658 ) =====
===== Goal reached (episode 659 ) =====
===== Goal reached (episode 661 ) =====
===== Goal reached (episode 662 ) =====
===== Goal reached (episode 663 ) =====
===== Goal reached (episode 664 ) =====
===== Goal reached (episode 666 ) =====
===== Goal reached (episode 667 ) =====
===== Goal reached (episode 668 ) =====
===== Goal reached (episode 669 ) =====
===== Goal reached (episode 670 ) =====
===== Goal reached (episode 671 ) =====
===== Goal reached (episode 672 ) =====
===== Goal reached (episode 673 ) =====
===== Goal reached (episode 674 ) =====
===== Goal reached (episode 675 ) =====
===== Goal reached (episode 676 ) =====


===== Goal reached (episode 1195 ) =====
===== Goal reached (episode 1196 ) =====
===== Goal reached (episode 1197 ) =====
===== Goal reached (episode 1199 ) =====
===== Goal reached (episode 1200 ) =====
===== Goal reached (episode 1201 ) =====
===== Goal reached (episode 1202 ) =====
===== Goal reached (episode 1203 ) =====
===== Goal reached (episode 1204 ) =====
===== Goal reached (episode 1205 ) =====
===== Goal reached (episode 1206 ) =====
===== Goal reached (episode 1207 ) =====
===== Goal reached (episode 1208 ) =====
===== Goal reached (episode 1209 ) =====
===== Goal reached (episode 1211 ) =====
===== Goal reached (episode 1212 ) =====
===== Goal reached (episode 1213 ) =====
===== Goal reached (episode 1214 ) =====
===== Goal reached (episode 1215 ) =====
===== Goal reached (episode 1216 ) =====
===== Goal reached (episode 1217 ) =====
===== Goal reached (episode 1218 ) =====
===== Goal reached (episode 1219 ) =====
===== Goal reached (episode 1220 ) =====
===== Goal reach

===== Goal reached (episode 1755 ) =====
===== Goal reached (episode 1756 ) =====
===== Goal reached (episode 1757 ) =====
===== Goal reached (episode 1758 ) =====
===== Goal reached (episode 1760 ) =====
===== Goal reached (episode 1763 ) =====
===== Goal reached (episode 1764 ) =====
===== Goal reached (episode 1765 ) =====
===== Goal reached (episode 1766 ) =====
===== Goal reached (episode 1767 ) =====
===== Goal reached (episode 1769 ) =====
===== Goal reached (episode 1770 ) =====
===== Goal reached (episode 1771 ) =====
===== Goal reached (episode 1772 ) =====
===== Goal reached (episode 1773 ) =====
===== Goal reached (episode 1774 ) =====
===== Goal reached (episode 1775 ) =====
===== Goal reached (episode 1776 ) =====
===== Goal reached (episode 1777 ) =====
===== Goal reached (episode 1779 ) =====
===== Goal reached (episode 1780 ) =====
===== Goal reached (episode 1781 ) =====
===== Goal reached (episode 1783 ) =====
===== Goal reached (episode 1784 ) =====
===== Goal reach

===== Goal reached (episode 2308 ) =====
===== Goal reached (episode 2309 ) =====
===== Goal reached (episode 2310 ) =====
===== Goal reached (episode 2311 ) =====
===== Goal reached (episode 2312 ) =====
===== Goal reached (episode 2313 ) =====
===== Goal reached (episode 2317 ) =====
===== Goal reached (episode 2318 ) =====
===== Goal reached (episode 2319 ) =====
===== Goal reached (episode 2320 ) =====
===== Goal reached (episode 2321 ) =====
===== Goal reached (episode 2322 ) =====
===== Goal reached (episode 2323 ) =====
===== Goal reached (episode 2324 ) =====
===== Goal reached (episode 2325 ) =====
===== Goal reached (episode 2326 ) =====
===== Goal reached (episode 2328 ) =====
===== Goal reached (episode 2329 ) =====
===== Goal reached (episode 2330 ) =====
===== Goal reached (episode 2331 ) =====
===== Goal reached (episode 2332 ) =====
===== Goal reached (episode 2333 ) =====
===== Goal reached (episode 2334 ) =====
===== Goal reached (episode 2335 ) =====
===== Goal reach

...

===== Goal reached (episode 7325 ) =====
===== Goal reached (episode 7326 ) =====
===== Goal reached (episode 7327 ) =====
===== Goal reached (episode 7328 ) =====
===== Goal reached (episode 7329 ) =====
===== Goal reached (episode 7330 ) =====
===== Goal reached (episode 7332 ) =====
===== Goal reached (episode 7333 ) =====
===== Goal reached (episode 7334 ) =====
===== Goal reached (episode 7335 ) =====
===== Goal reached (episode 7336 ) =====
===== Goal reached (episode 7337 ) =====
===== Goal reached (episode 7338 ) =====
===== Goal reached (episode 7339 ) =====
===== Goal reached (episode 7342 ) =====
===== Goal reached (episode 7344 ) =====
===== Goal reached (episode 7346 ) =====
===== Goal reached (episode 7347 ) =====
===== Goal reached (episode 7348 ) =====
===== Goal reached (episode 7349 ) =====
===== Goal reached (episode 7350 ) =====
===== Goal reached (episode 7351 ) =====
===== Goal reached (episode 7352 ) =====
===== Goal reached (episode 7353 ) =====
===== Goal reach

===== Goal reached (episode 7879 ) =====
===== Goal reached (episode 7880 ) =====
===== Goal reached (episode 7881 ) =====
===== Goal reached (episode 7884 ) =====
===== Goal reached (episode 7885 ) =====
===== Goal reached (episode 7886 ) =====
===== Goal reached (episode 7887 ) =====
===== Goal reached (episode 7888 ) =====
===== Goal reached (episode 7889 ) =====
===== Goal reached (episode 7891 ) =====
===== Goal reached (episode 7892 ) =====
===== Goal reached (episode 7893 ) =====
===== Goal reached (episode 7894 ) =====
===== Goal reached (episode 7895 ) =====
===== Goal reached (episode 7896 ) =====
===== Goal reached (episode 7897 ) =====
===== Goal reached (episode 7898 ) =====
===== Goal reached (episode 7899 ) =====
===== Goal reached (episode 7900 ) =====
===== Goal reached (episode 7901 ) =====
===== Goal reached (episode 7902 ) =====
===== Goal reached (episode 7904 ) =====
===== Goal reached (episode 7905 ) =====
===== Goal reached (episode 7906 ) =====
===== Goal reach

===== Goal reached (episode 8444 ) =====
===== Goal reached (episode 8445 ) =====
===== Goal reached (episode 8446 ) =====
===== Goal reached (episode 8447 ) =====
===== Goal reached (episode 8448 ) =====
===== Goal reached (episode 8449 ) =====
===== Goal reached (episode 8450 ) =====
===== Goal reached (episode 8452 ) =====
===== Goal reached (episode 8453 ) =====
===== Goal reached (episode 8454 ) =====
===== Goal reached (episode 8455 ) =====
===== Goal reached (episode 8456 ) =====
===== Goal reached (episode 8457 ) =====
===== Goal reached (episode 8458 ) =====
===== Goal reached (episode 8459 ) =====
===== Goal reached (episode 8462 ) =====
===== Goal reached (episode 8463 ) =====
===== Goal reached (episode 8465 ) =====
===== Goal reached (episode 8467 ) =====
===== Goal reached (episode 8468 ) =====
===== Goal reached (episode 8470 ) =====
===== Goal reached (episode 8471 ) =====
===== Goal reached (episode 8472 ) =====
===== Goal reached (episode 8473 ) =====
===== Goal reach

===== Goal reached (episode 9003 ) =====
===== Goal reached (episode 9005 ) =====
===== Goal reached (episode 9006 ) =====
===== Goal reached (episode 9007 ) =====
===== Goal reached (episode 9008 ) =====
===== Goal reached (episode 9011 ) =====
===== Goal reached (episode 9012 ) =====
===== Goal reached (episode 9013 ) =====
===== Goal reached (episode 9014 ) =====
===== Goal reached (episode 9016 ) =====
===== Goal reached (episode 9017 ) =====
===== Goal reached (episode 9018 ) =====
===== Goal reached (episode 9019 ) =====
===== Goal reached (episode 9021 ) =====
===== Goal reached (episode 9022 ) =====
===== Goal reached (episode 9023 ) =====
===== Goal reached (episode 9024 ) =====
===== Goal reached (episode 9025 ) =====
===== Goal reached (episode 9026 ) =====
===== Goal reached (episode 9027 ) =====
===== Goal reached (episode 9028 ) =====
===== Goal reached (episode 9029 ) =====
===== Goal reached (episode 9030 ) =====
===== Goal reached (episode 9031 ) =====
===== Goal reach

===== Goal reached (episode 9551 ) =====
===== Goal reached (episode 9552 ) =====
===== Goal reached (episode 9553 ) =====
===== Goal reached (episode 9554 ) =====
===== Goal reached (episode 9556 ) =====
===== Goal reached (episode 9557 ) =====
===== Goal reached (episode 9558 ) =====
===== Goal reached (episode 9560 ) =====
===== Goal reached (episode 9561 ) =====
===== Goal reached (episode 9562 ) =====
===== Goal reached (episode 9563 ) =====
===== Goal reached (episode 9566 ) =====
===== Goal reached (episode 9567 ) =====
===== Goal reached (episode 9568 ) =====
===== Goal reached (episode 9569 ) =====
===== Goal reached (episode 9570 ) =====
===== Goal reached (episode 9571 ) =====
===== Goal reached (episode 9572 ) =====
===== Goal reached (episode 9573 ) =====
===== Goal reached (episode 9574 ) =====
===== Goal reached (episode 9575 ) =====
===== Goal reached (episode 9576 ) =====
===== Goal reached (episode 9577 ) =====
===== Goal reached (episode 9579 ) =====
===== Goal reach